camera calibration with planar calibration target using the method described by Zhang

In [2]:
# load the images
import pathlib

image_dir = "calibration_images"

all_images = pathlib.Path(image_dir).glob('*.png')
all_images = [str(path) for path in all_images]

print(all_images)



['calibration_images/23.png', 'calibration_images/57.png', 'calibration_images/81.png', 'calibration_images/40.png', 'calibration_images/49.png', 'calibration_images/70.png', 'calibration_images/34.png', 'calibration_images/95.png', 'calibration_images/30.png', 'calibration_images/28.png', 'calibration_images/85.png', 'calibration_images/0.png', 'calibration_images/32.png', 'calibration_images/66.png', 'calibration_images/13.png', 'calibration_images/53.png', 'calibration_images/97.png', 'calibration_images/11.png', 'calibration_images/33.png', 'calibration_images/65.png', 'calibration_images/73.png', 'calibration_images/6.png', 'calibration_images/56.png', 'calibration_images/10.png', 'calibration_images/76.png', 'calibration_images/99.png', 'calibration_images/60.png', 'calibration_images/25.png', 'calibration_images/45.png', 'calibration_images/84.png', 'calibration_images/18.png', 'calibration_images/91.png', 'calibration_images/8.png', 'calibration_images/3.png', 'calibration_imag

In [3]:
# for all images, load the image and detect the marker corners using opencv

import cv2
import cv2.aruco
import numpy as np
np.set_printoptions(precision=2, suppress=True)


rejected_images = []

# see the pdf file in the airo-camera-toolkit/docs folder
aruco_dict =  cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)
charuco_board = cv2.aruco.CharucoBoard((7, 5), 0.04, 0.031, aruco_dict)
corner_dict = {}

for image in all_images:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, aruco_dict)

    n_corners, charuco_corners, ids = cv2.aruco.interpolateCornersCharuco(corners, ids, gray, charuco_board)
    if n_corners > 0:
        # refine corners
        charuco_corners = cv2.cornerSubPix(gray, charuco_corners, winSize=(3, 3), zeroZone=(-1, -1), criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01))

    if not len(charuco_corners) == 24:
        print(f"Error: {image} has {len(charuco_corners)} corners")
        rejected_images.append(image)
    else:
        corner_dict[image] = {"corners": corners, "charuco_corners": charuco_corners, "ids": ids}

print(f"Rejected images: {rejected_images}")
all_images = [image for image in all_images if image not in rejected_images]

# visualize the detected corners
import matplotlib.pyplot as plt

for image in all_images:
    img = cv2.imread(image)
    corners = corner_dict[image]
    img = cv2.aruco.drawDetectedCornersCharuco(img, np.array(corners["charuco_corners"]), np.array(corners["ids"]), (0, 255, 0))

    # convert from BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #plt.imshow(img)
    plt.show()

Error: calibration_images/34.png has 22 corners
Error: calibration_images/10.png has 22 corners
Error: calibration_images/92.png has 22 corners
Error: calibration_images/9.png has 22 corners
Error: calibration_images/98.png has 22 corners
Rejected images: ['calibration_images/34.png', 'calibration_images/10.png', 'calibration_images/92.png', 'calibration_images/9.png', 'calibration_images/98.png']


In [4]:
# create real-world planar coordinates. points are in a 4x6 grid with 40mm spacing 

# create the planar coordinates
planar_coords = []
for j in range(4):
    for i in range(6):
        planar_coords.append([i*0.02, j*0.02, 0])
print(planar_coords)



[[0.0, 0.0, 0], [0.02, 0.0, 0], [0.04, 0.0, 0], [0.06, 0.0, 0], [0.08, 0.0, 0], [0.1, 0.0, 0], [0.0, 0.02, 0], [0.02, 0.02, 0], [0.04, 0.02, 0], [0.06, 0.02, 0], [0.08, 0.02, 0], [0.1, 0.02, 0], [0.0, 0.04, 0], [0.02, 0.04, 0], [0.04, 0.04, 0], [0.06, 0.04, 0], [0.08, 0.04, 0], [0.1, 0.04, 0], [0.0, 0.06, 0], [0.02, 0.06, 0], [0.04, 0.06, 0], [0.06, 0.06, 0], [0.08, 0.06, 0], [0.1, 0.06, 0]]


In [5]:
# for each image, find the homography

homography_dict = {}

for image in all_images:
    corners = corner_dict[image]
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # make the corners homogeneous
    img_corners  = np.array(corners["charuco_corners"])
    # drop second dimension
    img_corners = img_corners.reshape(-1,2)
    img_corners = np.hstack([img_corners, np.ones((img_corners.shape[0],1))])

    # make world coords XY homogeneous
    p = np.array(planar_coords)
    p[:,2] = 1
    print(p.shape)
    print(img_corners.shape)
    

    # find the homography
    H,retval = cv2.findHomography(p, img_corners)
    homography_dict[image] = H
    print(H)

    # check 
    # get reprojection error

    # project the planar coordinates to the image
    img_coords = np.dot(H, p.T)
    img_coords = img_coords/img_coords[2,:]
    img_coords = img_coords[:2,:].T
    img_coords = img_coords.reshape(-1,2)

    l2_error = np.linalg.norm(img_coords - img_corners[:,:2])
    l2_error = l2_error/img_coords.shape[0]
    print(l2_error)



(24, 3)
(24, 3)
[[ 3869.44 -1135.57   613.85]
 [ -828.47  3720.01   314.  ]
 [   -0.73    -1.19     1.  ]]
0.11784916244105968
(24, 3)
(24, 3)
[[2171.56 -706.32  777.42]
 [2030.85 3969.14  261.71]
 [  -0.91    1.31    1.  ]]
0.10465036419079847
(24, 3)
(24, 3)
[[ 3686.87 -1689.47   810.41]
 [  467.86   155.98   558.78]
 [    0.45    -2.2      1.  ]]
0.05189518777273372
(24, 3)
(24, 3)
[[ 2756.35 -1991.5    718.16]
 [ -342.05  -236.42   675.26]
 [   -0.06    -2.01     1.  ]]
0.03256096390019723
(24, 3)
(24, 3)
[[3112.75  175.7   198.57]
 [ 457.4  3574.37  296.8 ]
 [  -0.32    0.67    1.  ]]
0.10562129223208877
(24, 3)
(24, 3)
[[ 2960.16 -2304.15   806.79]
 [-1781.33  1198.68   597.44]
 [   -0.79    -2.37     1.  ]]
0.04282764258482543
(24, 3)
(24, 3)
[[14535.04  3068.16   768.15]
 [  362.99 13060.31   248.86]
 [    3.72     4.87     1.  ]]
0.24269716281266926
(24, 3)
(24, 3)
[[ 491.18 -868.17 1308.47]
 [ 315.18 3789.7   281.58]
 [  -1.95    0.48    1.  ]]
0.1161861818654338
(24, 3)
(24,

In [24]:
# calibrate camera using zhang method algebraic solution


def v_ij(H,i,j):
    row = [H[i,0]*H[j,0], H[i,0]*H[j,1] + H[i,1]*H[j,0], H[i,1]*H[j,1], H[i,2]*H[j,0] + H[i,0]*H[j,2], H[i,2]*H[j,1] + H[i,1]*H[j,2], H[i,2]*H[j,2]]
    return np.array(row)


V = np.zeros((2*len(all_images), 6))
for i,H in enumerate(homography_dict.values()):
    V[2*i] = v_ij(H,0,1)
    V[2*i+1] = v_ij(H,0,0) - v_ij(H,1,1)
U,S,Vt = np.linalg.svd(V)
b = Vt[-1]

B = np.array([[b[0], b[1], b[3]], [b[1], b[2], b[4]], [b[3], b[4], b[5]]])

v0 = (B[0,1]*B[0,2] - B[0,0]*B[1,2])/(B[0,0]*B[1,1] - B[0,1]*B[0,1])
l = B[2,2] - (B[0,2]*B[0,2] + v0*(B[0,1]*B[0,2] - B[0,0]*B[1,2]))/B[0,0]
alpha = np.sqrt(l/B[0,0])
beta = np.sqrt(l*B[0,0]/(B[0,0]*B[1,1] - B[0,1]*B[0,1]))
gamma = -B[0,1]*alpha*alpha*beta/l
u0 = gamma*v0/alpha - B[0,2]*alpha*alpha/l


K = np.array([[alpha, gamma, u0], [0, beta, v0], [0, 0, 1]])
print(K)
                        

[[ 5.394 -0.349  8.537]
 [ 0.     6.358 -0.356]
 [ 0.     0.     1.   ]]


In [21]:
# use opencv to calibrate
from random import sample
objPoints = []
imgPoints = []
for img in sample(all_images, 95):
    corners = corner_dict[img]
    objPoints.append(np.array(planar_coords))
    imgPoints.append(corners["charuco_corners"])

objPoints = np.array(objPoints)
print(objPoints.shape)
imgPoints = np.array(imgPoints)
imgPoints = imgPoints.reshape(imgPoints.shape[0],imgPoints.shape[1],2)
print(imgPoints.shape)
imgPoints = imgPoints.astype(np.float32)
objPoints = objPoints.astype(np.float32)

# no distortion optimization!
retval, cameraMatrix, distCoeffs, rvecs, tvecs, std_dev_intrinsics,std_dev_extrinsics,per_view_rms = cv2.calibrateCameraExtended(objPoints, imgPoints, gray.shape[::-1], None, np.zeros(5)) #, flags=cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K2 | cv2.CALIB_FIX_K1 | cv2.CALIB_FIX_TANGENT_DIST)

np.set_printoptions(precision=3)
print(cameraMatrix)

print(distCoeffs)




(95, 24, 3)
(95, 24, 2)
[[1363.707    0.     959.372]
 [   0.    1364.964  561.763]
 [   0.       0.       1.   ]]
[ 0.196 -0.644 -0.001 -0.001  0.592]


In [22]:
print(np.average(per_view_rms))

0.634835554538635


In [9]:
import pyrealsense2 as rs

# shut camera down if it is running
ctx = rs.context()
devices = ctx.query_devices()
for dev in devices:
    dev.hardware_reset()

config = rs.config()

config.enable_stream(rs.stream.color, 1920,1080, rs.format.rgb8, 15)

# Avoid having to reconnect the USB cable, see https://github.com/IntelRealSense/librealsense/issues/6628#issuecomment-646558144
ctx = rs.context()
devices = ctx.query_devices()
for dev in devices:
    dev.hardware_reset()

pipeline = rs.pipeline()

pipeline.start(config)
# Get intrinsics matrix
profile = pipeline.get_active_profile()
color_profile = rs.video_stream_profile(profile.get_stream(rs.stream.color))
intrinsics = color_profile.get_intrinsics()
intrinsics_matrix = np.array(
    [
        [intrinsics.fx, 0, intrinsics.ppx],
        [0, intrinsics.fy, intrinsics.ppy],
        [0, 0, 1],
    ]
)

print(intrinsics_matrix)

[[1372.015    0.     967.68 ]
 [   0.    1369.215  558.417]
 [   0.       0.       1.   ]]
